In [1]:
import sys
get_ipython().system('{sys.executable} -m pip install kafka flair nltk torch==1.6')

     |████████████████████████████████| 215kB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 450kB 12.0MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 748.8MB 98.3MB/s eta 0:00:01                        | 77.4MB 82.3MB/s eta 0:00:09     |███▍                            | 80.5MB 82.3MB/s eta 0:00:09     |█████▏                          | 122.1MB 82.3MB/s eta 0:00:08��████▊                       | 203.8MB 84.6MB/s eta 0:00:07     |█████████▉                      | 231.1MB 78.2MB/s eta 0:00:07�█▏                     | 238.0MB 78.2MB/s eta 0:00:07MB/s eta 0:00:04��██▏           | 471.9MB 78.5MB/s eta 0:00:04:00:031.0MB 66.4MB/s eta 0:00:03███████████████████████▏      | 588.6MB 66.4MB/s eta 0:00:03███████████████████████▍      | 595.1MB 66.4MB/s eta 0:00:03�████     | 631.2MB 79.4MB/s eta 0:00:02��████████████████████████▉   | 674.8MB 79.4MB/s eta 0:00:01██▎ | 707.5MB 65.3MB/s eta 0:00:01�██▍| 733.6MB 

In [2]:
import flair
from kafka import KafkaConsumer
from kafka import KafkaProducer
import nltk
import json

In [15]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

2021-02-02 15:43:07,731 loading file /opt/app-root/src/.flair/models/sentiment-en-mix-distillbert_3.1.pt


[nltk_data] Downloading package punkt to /opt/app-
[nltk_data]     root/src/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /opt/app-root/src/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [16]:
brokers = 'odh-message-bus-kafka-brokers:9092'
from_topic = 'decoded-speech'
to_topic = 'sentiment-text'

consumer = KafkaConsumer(from_topic, bootstrap_servers=brokers)
producer = KafkaProducer(bootstrap_servers=brokers)

Exception ignored in: <bound method tqdm.__del__ of Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]>
Traceback (most recent call last):
  File "/opt/app-root/lib/python3.6/site-packages/tqdm/std.py", line 1134, in __del__
    self.close()
  File "/opt/app-root/lib/python3.6/site-packages/tqdm/notebook.py", line 280, in close
    self.disp(bar_style='danger')
AttributeError: 'tqdm' object has no attribute 'disp'


In [39]:
for msg in consumer:
    if msg.value is not None:
        obj_in = json.loads(msg.value.decode('utf-8'))
        
        # sentiment
        s = flair.data.Sentence(obj_in["sentence"])
        flair_sentiment.predict(s)
        
        # nltk nouns
        text = nltk.word_tokenize(obj_in["sentence"])
        tokens = nltk.pos_tag(text)
        nouns = []
        for pair in tokens:
            if pair[1][:2] == 'NN':
                nouns.append(pair[0])
        
        data = {
            "sentence": obj_in["sentence"],
            "quality": s.labels[0].value,
            "nouns": nouns,
            "id": obj_in["id"]
        }
        print(data)
        
        producer.send(to_topic, json.dumps(data).encode('utf-8'))

print('exiting')


{'sentence': "thank you mrs been very helpful i think i'll give you a five star review on your support page", 'quality': 'POSITIVE', 'nouns': ['thank', 'i', 'star', 'review', 'support', 'page'], 'id': -7521911946240623369}
{'sentence': "i can't believe the done to my new cars not covered under warranty how could this not be covered under warranty cuyp we speak your supervisor right now", 'quality': 'NEGATIVE', 'nouns': ['i', 'cars', 'warranty', 'warranty', 'cuyp', 'supervisor'], 'id': -7521911946240623369}
{'sentence': "i can't believe the done to my new cars not covered under warranty how could this not be covered under warranty cuyp we speak your supervisor right now", 'quality': 'NEGATIVE', 'nouns': ['i', 'cars', 'warranty', 'warranty', 'cuyp', 'supervisor'], 'id': -7521911946240623369}
{'sentence': "i don't have time to talk to you about this honey my car fixed right now without fixed right now i'll have to be going to another dealership", 'quality': 'NEGATIVE', 'nouns': ['i', 'tim

KeyboardInterrupt: 